Inspired by [assistant-wind_farm.ipynb](https://github.com/Azure-Samples/azureai-samples/blob/main/scenarios/Assistants/api-in-a-box/wind_farm/assistant-wind_farm.ipynb)

In [1]:
# Define the Assistant tools

tools_list = [
    {"type": "code_interpreter"},
]

In [2]:
# Libraries and Constants

import os
from dotenv import load_dotenv # requires python-dotenv

load_dotenv("./../credentials_my.env")
MODEL = os.environ["GPT4-1106-128k"]
should_cleanup = True

In [3]:
from openai import AzureOpenAI

In [4]:
# Create the client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview", # at least 2024-02-15-preview
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [5]:
# Upload the Assistant file(s)

from openai.types import FileObject
from pathlib import Path

DATA_FOLDER = "./data/"

def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")

arr = os.listdir(DATA_FOLDER)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    print(f"Uploading {filePath}...")
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]
assistant_files

Uploading ./data/turbines.csv...


[FileObject(id='assistant-AWlWyRx8YGLuNajWWlrlfVK8', bytes=576, created_at=1710845807, filename='turbines.csv', object='file', purpose='assistants', status='processed', status_details=None)]

In [6]:
# Create an Assistant and a Thread
assistant = client.beta.assistants.create(
    name="Portfolio Management Assistant",
    instructions="You are an assistant that can help manage wind turbine farm. "
    + "The turbines operating ranges are output voltages of 33kv-35kv and RPM of 15-25. Wind speed is measured in miles per hour."
    + "Maintenance should occur every 12 months. Greet the user by saying, 'Welcome Turbine Management Assistant.'",
    tools=tools_list,
    model=MODEL,
    file_ids=file_ids,
)

thread = client.beta.threads.create()

print(assistant.model_dump_json(indent=2))

{
  "id": "asst_LbLtP5Bqks01qucrlq00clwj",
  "created_at": 1710845809,
  "description": null,
  "file_ids": [
    "assistant-AWlWyRx8YGLuNajWWlrlfVK8"
  ],
  "instructions": "You are an assistant that can help manage wind turbine farm. The turbines operating ranges are output voltages of 33kv-35kv and RPM of 15-25. Wind speed is measured in miles per hour.Maintenance should occur every 12 months. Greet the user by saying, 'Welcome Turbine Management Assistant.'",
  "metadata": {},
  "model": "gpt4-1106-128k",
  "name": "Portfolio Management Assistant",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ]
}


In [7]:
def process_message(content: str) -> None:
    from datetime import datetime
    import time, json
    
    start_time = time.time()
    
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content)

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="The current date and time is: " + datetime.now().strftime("%x %X") + ". ",
    )

    print("Processing...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed" or run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            print(json.loads(messages.json())["data"][0]["content"][0]["text"]["value"])            
            break
        if run.status == "expired":
            # Handle expired
            break
        if run.status == "cancelled":
            # Handle cancelled
            break
        if run.status == "requires_action":
            pass
        else:
            print(f"Run status: {run.status} after {int((time.time() - start_time) // 60)} minutes {int((time.time() - start_time) % 60)} seconds")
            time.sleep(5)
            
    return messages

In [8]:
r1 = process_message("What is the status of turbine 1001 and 1003?")

Processing...
Run status: queued after 0 minutes 0 seconds
Run status: in_progress after 0 minutes 6 seconds
Run status: in_progress after 0 minutes 11 seconds
Run status: in_progress after 0 minutes 16 seconds
Run status: in_progress after 0 minutes 21 seconds
Run status: in_progress after 0 minutes 27 seconds
Run status: in_progress after 0 minutes 32 seconds
Run status: in_progress after 0 minutes 37 seconds
Run status: in_progress after 0 minutes 42 seconds
Run status: in_progress after 0 minutes 48 seconds
Run status: in_progress after 0 minutes 53 seconds
Run status: in_progress after 0 minutes 58 seconds
Run status: in_progress after 1 minutes 3 seconds
Run status: in_progress after 1 minutes 9 seconds
Run status: in_progress after 1 minutes 14 seconds
Run status: in_progress after 1 minutes 19 seconds
Run status: in_progress after 1 minutes 24 seconds
Run status: in_progress after 1 minutes 30 seconds
Run status: in_progress after 1 minutes 35 seconds
Run status: in_progress af

In [9]:
r2 = process_message("Show me the code you used to provide the previous answer")

Processing...
Run status: queued after 0 minutes 0 seconds
Run status: in_progress after 0 minutes 6 seconds
Run status: in_progress after 0 minutes 11 seconds
Run status: in_progress after 0 minutes 16 seconds
Run status: in_progress after 0 minutes 22 seconds
Run status: in_progress after 0 minutes 27 seconds
Certainly! Below is the Python code snippet that I used to parse the CSV data and extract the status for turbines 1001 and 1003:

```python
import pandas as pd
from io import StringIO

# Create a String buffer to simulate a file-like object for the CSV data
data = StringIO(content_or_error)
# Load the data into a pandas DataFrame
df = pd.read_csv(data)

# Filter the DataFrame for the status of turbines 1001 and 1003
turbine_statuses = df[df['Turbine_ID'].isin([1001, 1003])]
```

The code does the following steps:

1. Import the required `pandas` module and `StringIO` class.
2. Use `StringIO` to create a file-like object from the loaded CSV content (which I read as a plain text i

In [10]:
# Cleaning up

if should_cleanup:
    client.beta.assistants.delete(assistant.id)
    client.beta.threads.delete(thread.id)
    for file in client.files.list():
        print(f"Deleting the file {file.filename} (id={file.id})")
        client.files.delete(file.id)

Deleting the file turbines.csv (id=assistant-AWlWyRx8YGLuNajWWlrlfVK8)
